In [1]:
import os
import zipfile
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

dataset_path = "/kaggle/input/fer2013-cleaned-dataset/images1"

num_classes = 7

input_shape = (48, 48, 1)

batch_size = 64


In [3]:
from keras.models import Sequential
import numpy as np
import tensorflow as tf
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization


model = Sequential()


model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(7, activation='softmax'))


model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 48, 48, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 48, 48, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 32)        0         
 D)                                                              
                                                        

In [5]:
train_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

train_data = train_data_generator.flow_from_directory(
    os.path.join(dataset_path, 'train'),
    color_mode='grayscale',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_data = test_data_generator.flow_from_directory(
    os.path.join(dataset_path, 'test'),
    color_mode='grayscale',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 28044 images belonging to 7 classes.
Found 7177 images belonging to 7 classes.


In [6]:
num_train_samples = 28044
num_test_samples = 7177

In [7]:
from keras.optimizers import Adam, RMSprop, SGD, Adagrad,Nadam

In [8]:


# Compile the model with Nadam optimizer and the specified learning rate
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])


In [9]:
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=7)


In [11]:
history = model.fit(
    train_data,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=10,
    callbacks=[early_stopping],
    batch_size=batch_size,
    validation_data=test_data,  # Add validation data
    validation_steps=num_test_samples // batch_size  # Specify validation steps
)


Epoch 1/10


2024-01-10 04:14:54.509802: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


438/438 [==============================] - 210s 461ms/step - loss: 1.9328 - accuracy: 0.3380 - val_loss: 1.8110 - val_accuracy: 0.3400
Epoch 2/10
438/438 [==============================] - 29s 66ms/step - loss: 1.4421 - accuracy: 0.4575 - val_loss: 1.5767 - val_accuracy: 0.4198
Epoch 3/10
438/438 [==============================] - 29s 66ms/step - loss: 1.2814 - accuracy: 0.5130 - val_loss: 1.2537 - val_accuracy: 0.5321
Epoch 4/10
438/438 [==============================] - 29s 67ms/step - loss: 1.1987 - accuracy: 0.5428 - val_loss: 1.3721 - val_accuracy: 0.4819
Epoch 5/10
438/438 [==============================] - 29s 67ms/step - loss: 1.1460 - accuracy: 0.5659 - val_loss: 1.1625 - val_accuracy: 0.5589
Epoch 6/10
438/438 [==============================] - 29s 67ms/step - loss: 1.0871 - accuracy: 0.5888 - val_loss: 1.1576 - val_accuracy: 0.5647
Epoch 7/10
438/438 [==============================] - 29s 66ms/step - loss: 1.0520 - accuracy: 0.6053 - val_loss: 1.1465 - val_accuracy: 0.5763
E

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [13]:
# Evaluate the model on the test data
y_true = test_data.classes  # Assuming you're using flow_from_directory
predictions = model.predict(test_data)

# Convert predictions to class labels
y_pred = np.argmax(predictions, axis=1)

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# If binary classification, compute AUC  # AUC is not applicable for multi-class

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)




113/113 [==============================] - 6s 50ms/step
Accuracy: 0.5985787933677024
Precision: 0.6086648061780819
Recall: 0.5985787933677024
F1 Score: 0.5854420721146442


In [14]:
test_loss, test_accuracy = model.evaluate(
    test_data,
    steps=num_test_samples // batch_size
)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

112/112 [==============================] - 6s 52ms/step - loss: 1.0950 - accuracy: 0.5989
Test Loss: 1.0949565172195435
Test Accuracy: 0.5989118218421936


In [15]:
from sklearn.preprocessing import StandardScaler
from keras.models import Model

# Extract features from the last convolutional layer
conv_output = model.get_layer('conv2d_5').output  # Adjust the layer name based on your model summary
feature_extraction_model = Model(inputs=model.input, outputs=conv_output)

# Get features for the training data
train_features = feature_extraction_model.predict(train_data)

# Reshape the features to be 1D arrays for each sample
train_features_1d = train_features.reshape((num_train_samples, -1))

# Get features for the test data
test_features = feature_extraction_model.predict(test_data)

# Reshape the features to be 1D arrays for each sample
test_features_1d = test_features.reshape((num_test_samples, -1))

# Preprocess the features using StandardScaler
scaler = StandardScaler()

# Fit on training data
scaler.fit(train_features_1d)

# Transform training and test features
train_features_scaled = scaler.transform(train_features_1d)
test_features_scaled = scaler.transform(test_features_1d)


113/113 [==============================] - 6s 54ms/step


In [16]:
from sklearn.decomposition import KernelPCA, IncrementalPCA
from sklearn.model_selection import train_test_split
subset_size = 5000
train_features_scaled_subset, _, train_data_subset, _ = train_test_split(
    train_features_scaled, train_data.classes, train_size=subset_size, random_state=42
)

# Batch processing for faster experimentation
batch_size = 1000
num_batches = len(train_features_scaled_subset) // batch_size

# Apply Incremental PCA for dimensionality reduction
n_components = 50
ipca = IncrementalPCA(n_components=n_components, batch_size=batch_size)

# Process data in batches
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size

    train_features_kpca_batch = ipca.fit_transform(train_features_scaled_subset[start_idx:end_idx])

# Transform the test data using IPCA
test_features_kpca_ipca = ipca.transform(test_features_scaled)

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.svm import SVC, NuSVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Define a list of classifiers
classifiers = [
    ('KNN', KNeighborsClassifier()),
    ('LDA', LinearDiscriminantAnalysis()),
    ('RF', RandomForestClassifier()),
    ('AdaBoost', AdaBoostClassifier()),
    ('ETC', ExtraTreesClassifier()),
    ('XBC', XGBClassifier()),
    ('SVM', SVC()),
    ('Bagging', BaggingClassifier()),
    
    ('GBC', GradientBoostingClassifier())
]

# Train and evaluate each classifier
for name, classifier in classifiers:
    # Use the full training dataset
    classifier.fit(train_features_kpca_batch, train_data_subset[:batch_size])
    predictions = classifier.predict(test_features_kpca_ipca)
    accuracy = accuracy_score(test_data.classes, predictions)
    print(f"{name} Accuracy on the test set: {accuracy * 100:.2f}%")

# Create a stacked ensemble model
stacked_model = StackingClassifier(
    estimators=classifiers,
    final_estimator=LogisticRegression()
)

# Train the stacked ensemble model
stacked_model.fit(train_features_kpca_batch, train_data_subset[:batch_size])

# Make predictions on the test set using the stacked model
stacked_predictions = stacked_model.predict(test_features_kpca_ipca)

# Evaluate the accuracy of the stacked model
stacked_accuracy = accuracy_score(test_data.classes, stacked_predictions)
print(f"Stacked Model Accuracy on the test set: {stacked_accuracy * 100:.2f}%")

KNN Accuracy on the test set: 17.42%
LDA Accuracy on the test set: 17.88%
RF Accuracy on the test set: 20.96%
AdaBoost Accuracy on the test set: 17.40%
ETC Accuracy on the test set: 20.90%
XBC Accuracy on the test set: 18.42%
SVM Accuracy on the test set: 19.20%
Bagging Accuracy on the test set: 18.85%
GBC Accuracy on the test set: 19.02%


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Stacked Model Accuracy on the test set: 21.55%
